---
# Защита данных клиентов страховой компании

---

## Описание проекта

<u>**Цель проекта**</u>: Разработка метода защиты персональных данных клиентов банка;

<u>**Сферы деятельности компаний**</u>: Банковская сфера;

<u>**Навыки и инструменты**</u>: Линейная алгебра &#128293;

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import math as m
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 12, 10 #закомментировать в случае возникновения проблем :)
sns.set(rc={'figure.figsize':(14,10)})

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.metrics import r2_score

import joblib

In [5]:
data_insurance = pd.read_csv(...)

In [6]:
display(data_insurance.head(30))
display(data_insurance.info())
display(data_insurance.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $X'$ — изменённая матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $w'$ — изменённый вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

#### "Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?"

**Ответ:** b. При умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

**Обоснование:** 

$$
a = Xw
$$

$$
a' = (XP)w'
$$

$$
w' = (X'^T X')^{-1} X'^T y
$$

$$
X' = XP
$$

$$
w' = [(XP)^T (XP)]^{-1} (XP)^T y
$$

$$
w = (X^T X)^{-1} X^T y
$$

$$
w' = w P^-1
$$

Видно, что старые весовые коэффициенты $w$ коррелируют с новыми коэффициентами $w'$, полученными после умножения на случайную обратимую матрицу $P$.

## 3. Алгоритм преобразования

**Алгоритм:**

Используем полученные ранее результаты в качестве алгоритма преобразования данных:

Умножим матрицу признаков $X$ на обратимую матрицу $P$. Умножение можно повторять последовательно сколь угодно много раз, однако это лишь увеличит время шифрования/дешифрования без повышения надёжности хранения данных.

**Обоснование:**

$$
a' = (XP)w
$$

$$
a'P^-1=(XP)wP^-1=X(PP^-1)w=XEw=Xw=a
$$

## 4. Проверка алгоритма

Исследуем **изменение качества линейной регрессии при умножении признаков на обратимую матрицу**.

Для этого **выделим целевой признак** и **выполним разделение общей выборки на обучающую и валидационную**:

In [7]:
features = data_insurance.drop('Страховые выплаты', axis=1)
target = data_insurance['Страховые выплаты']

features_train, features_valid, target_train, target_valid  = train_test_split(features,
                                                                               target,
                                                                               test_size=0.25,
                                                                               random_state=12345)

Обучим линейную регрессию и получим коэффициент детерминации R²:

In [8]:
lr_predictions = (LinearRegression()
      .fit(features_train, target_train)
      .predict(features_valid))
lr_r2 = r2_score(target_valid, lr_predictions)
print('Коэффициент детерминации: {:.4f}'.format(lr_r2))

Коэффициент детерминации: 0.4352


**Сгенерируем обратимую матрицу $P$, проверим её обратимость**:

In [9]:
P = np.random.randint(0, 1000, size=(4, 4))
P_inv = np.linalg.inv(P)
print('Сгенерированная матрица:')
display(P)
print()
print()
print()
print('Обратная сгенерированной матрица:')
display(P_inv)

Сгенерированная матрица:


array([[362,  38, 149,   4],
       [762,  97,  74, 158],
       [103, 434, 783, 774],
       [ 45, 665, 660, 749]])




Обратная сгенерированной матрица:


array([[ 0.00055454,  0.00107037, -0.00010752, -0.00011765],
       [ 0.00211531, -0.00075164, -0.00353322,  0.00379842],
       [ 0.00499411, -0.00248376,  0.00110404, -0.00064362],
       [-0.00631208,  0.00279166,  0.00217058, -0.00146311]])

Зашифруем признаки из обучающей и валидационной выборок умножением на случайную обратимую матрицу $P$:

In [10]:
features_train_encrypted = pd.DataFrame(features_train.values.dot(P).astype(np.int64))
features_valid_encrypted = pd.DataFrame(features_valid.values.dot(P).astype(np.int64))

display(features_train_encrypted)
display(features_valid_encrypted)

,0,1,2,3
0,3761773,15715674,28348591,28026347
1,5959070,25001736,45103465,44587776
2,4257729,17841169,32184328,31817205
3,4672777,19577557,35316624,34913837
4,5237036,21964931,39623562,39171112
...,...,...,...,...
3745,3338304,13935474,25137408,24852036
3746,2359616,9857176,17778812,17577220
3747,4635749,19404480,35003943,34605031
3748,5177244,21748194,39232568,38783872


,0,1,2,3
0,4042326,16931861,30542082,30194210
1,4477490,18711580,33752320,33368798
2,4366380,18275221,32967335,32591566
3,3599640,15105140,27249880,26938360
4,4151422,17366637,31325674,30969474
...,...,...,...,...
1245,3026314,12592315,22712243,22455560
1246,5989343,25133834,45340271,44822065
1247,4383932,18361633,33123639,32745734
1248,4281694,17927730,32340713,31971892


Успех! Догадаться, какие признаки и объекты содержались в выборках, уже невозможно.

Теперь убедимся в обратимости произошедших изменений. Умножим матрицу зашифрованных признаков на матрицу, обратную к случайной, справа.

In [11]:
pd.DataFrame(features_train_encrypted
             .values
             .dot(P_inv)
             .round()
             .astype(np.int64),
             columns=features_train.columns.tolist()
            ).head(30)

,Пол,Возраст,Зарплата,Члены семьи
0,1,43,36200,1
1,1,34,57600,0
2,0,32,41100,1
3,0,36,45100,1
4,0,33,50600,2
5,0,22,56800,1
6,1,20,48100,2
7,1,33,41100,1
8,1,42,46700,0
9,1,21,30500,1


**Признаки дешифрованы и имеют тот же вид, который они имели до преобразования**.

Сравним коэффициенты детерминации:

In [12]:
lr_predictions_encrypted = (LinearRegression()
      .fit(features_train_encrypted, target_train)
      .predict(features_valid_encrypted))
lr_r2_encrypted = r2_score(target_valid, lr_predictions_encrypted)
print('Коэффициент детерминации до шифрования:     {:.4f}'.format(lr_r2))
print('Коэффициент детерминации после шифрования:  {:.4f}'.format(lr_r2_encrypted))

Коэффициент детерминации до шифрования:     0.4352
Коэффициент детерминации после шифрования:  0.4352


Полученные результаты абсолютно закономерны в соответствии с определениями единичной и обратной матриц:

$$
a'P^-1=(XP)wP^-1=X(PP^-1)w=XEw=Xw=a
$$


<div style="border: solid darkcyan 3px; padding: 30px">
    <h3 style="color:black; margin-bottom:25px"> Выводы: </h3>
    <ul>
        <li><b>Загружен и изучен датасет</b>;</li>
        <li><b>Доказана неизменность качества обучения линейной регрессии при умножении матрицы признаков на случайную обратимую матрицу</b>;</li>
        <li><b>Предложен и описан алгоритм шифрования/дешифрования данных, доказана возможность стопроцентного дешифрования при работе с целочисленными признаками</b>;</li>
        <li><b>Алгоритм реализован, произведена оценка качества с применением коэффициентов детерминации</b>.</li>
    </ul>
</div>